# Fake News Detector

### Alex Hedrick

#### About the data (downloaded from Kaggle):

(WELFake) is a dataset of 72,134 news articles with 35,028 real and 37,106 fake news. For this, authors merged four popular news datasets (i.e. Kaggle, McIntire, Reuters, BuzzFeed Political) to prevent over-fitting of classifiers and to provide more text data for better ML training.

Published in:
IEEE Transactions on Computational Social Systems: pp. 1-13 (doi: 10.1109/TCSS.2021.3068519).

0 = fake, 1 = real

In [7]:
# import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline
import nltk

# read data
news = pd.read_csv('news_data_2_recent/WELFake_Dataset.csv', sep=',', names = ['index','title','body','label'], header = None, skiprows = 1)
news = news.drop('index', axis=1)
news.head()

,title,body,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


#### Data Preprocessing

In [11]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# remove all non-alphanumeric characters from news
news['title'] = news['title'].str.replace('[^a-zA-Z0-9]', ' ')
news['body'] = news['body'].str.replace('[^a-zA-Z0-9]', ' ')

# convert all news to lowercase
news['title'] = news['title'].str.lower()
news['body'] = news['body'].str.lower()

# word tokenize news for lemmatizer
news['title'] = news['title'].apply(nltk.word_tokenize)
news['body'] = news['body'].apply(nltk.word_tokenize)

# lemmatize news
lemmatizer = nltk.stem.WordNetLemmatizer()
news['title'] = news['title'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x]))
news['body'] = news['body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x]))


# # word tokenize news again
# # news['title'] = news['title'].apply(nltk.word_tokenize)
# # news['body'] = news['body'].apply(nltk.word_tokenize)

# # replace 0 with -1 for label
# # now, -1 = fake news, 1 = real news
# news['label'] = news['label'].map({0: -1})

news.head()

TypeError: 'float' object is not iterable